In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


from time import strftime, strptime, mktime, time, ctime

/home/shj16110/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
mktime?

In [2]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

In [3]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)

df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
s = StandardScaler()
df = s.fit_transform(df)
test = s.transform(test)

m = lambda *x: [i.shape for i in x]
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
print m(x_train, y_train, x_test, y_test)



(69713, 22)
(30037, 21)
[(55770, 37416), (55770,), (13943, 37416), (13943,)]


In [4]:
%%time
d_train = xgb.DMatrix(x_train, label=y_train)
d_test = xgb.DMatrix(x_test, label=y_test)


CPU times: user 19.8 s, sys: 9.08 s, total: 28.9 s
Wall time: 28.9 s


In [ ]:
RANDOM_STATE = 2020
params = {
    'min_child_weight': 0.9,
#     'eta': 0.1,
    'colsample_bytree': 0.9,
    'max_depth': 15,
    'subsample': 0.7,
#     'alpha': 0.3,
#     'lambda':0.9,
#     'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.03,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':24,
    'eval_metric':'auc'
    
}
watchlist = [(d_train, 'train'), (d_test, 'test')]



In [ ]:
start=time()

clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=10, 
                verbose_eval=1,  maximize=True,
               )
                


'!!! used %.2f s'%(time()-start)

In [11]:
clf

NameError: name 'clf' is not defined

In [ ]:
pred = model.predict(x_test)
predb = model.predict_proba(x_test)

prob = predb[:,1]
roc_auc_score(y_test, prob)


600 6 0.8224773007456205

600 5 std 0.8404221862156795


In [24]:
roc_auc_score(y_test, prob)

0.730066578200787

___

In [36]:
prob_s = prob_test[:,1]
prob_s

array([0.00386508, 0.04251419, 0.00330556, ..., 0.        , 0.00638889,
       0.03449272])

In [37]:
submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(prob_s, columns=['Approved'])], axis=1)
submit.shape

(30037, 2)

In [38]:
submit.to_csv('fr600.t5_f_std.csv',index=False)

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba